# Eval: Sentence Transformer with question-result set

This notebook outlines the steps undertook to evaluate various sentence transformer models out of the box with the test set under a QA set-up.

With 512 chunk size.

# Import libraries

In [2]:
#%pip install nltk -q
#%pip install sentence_transformers -q

import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

from sentence_transformers import SentenceTransformer, util

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# Global variable

In [3]:
EVALS = {}
MAX_WORDS = 512

# Helper functions

In [4]:
def split_text_into_chunks(text, max_words=MAX_WORDS, overlap_sentences=3):
    sentences = sent_tokenize(text)  # Tokenize into sentences
    chunks = []
    start_idx = 0  # Start index of the current chunk

    while start_idx < len(sentences):
        current_chunk = []
        current_word_count = 0
        idx = start_idx

        # Build a chunk until max_words is reached
        while idx < len(sentences) and current_word_count + len(sentences[idx].split()) <= max_words:
            current_chunk.append(sentences[idx])
            current_word_count += len(sentences[idx].split())
            idx += 1  # Move to next sentence

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        # Stop if the last chunk reaches the end of the text
        if idx >= len(sentences):
            break

        # Move start index forward but keep overlap
        start_idx = max(idx - overlap_sentences, start_idx + 1)

    return chunks


def create_embeddings(df, model, text_column, embedding_column):
    df[embedding_column] = None

    for idx, text in enumerate(df[text_column]):
        chunks = split_text_into_chunks(text)  # Chunk the text
        chunk_embeddings = np.array(model.encode(chunks))  # Encode all chunks (shape: [num_chunks, embedding_dim])
        df.at[idx, embedding_column] = chunk_embeddings

    return df


def evaluate_retrieval(query_emb, doc_embs, relevant_idx, top_k=5):
    # Flatten the document chunks and keep track of the document they belong to
    all_chunk_embeddings = []
    doc_chunk_mapping = []  # Mapping from chunk index to document index
    for doc_idx, doc_chunks in enumerate(doc_embs):
        for chunk_emb in doc_chunks:
            all_chunk_embeddings.append(chunk_emb)
            doc_chunk_mapping.append(doc_idx)  # Store the document index for each chunk

    # Convert to numpy array for efficient computation
    all_chunk_embeddings = np.array(all_chunk_embeddings)  # Shape: [total_chunks, 768]

    # Compute cosine similarity between query and all document chunks
    similarities = np.dot(all_chunk_embeddings, query_emb.T)  # Shape: [total_chunks,]

    # Get indices of top-k most similar chunks
    top_k_chunk_indices = np.argsort(similarities.flatten())[::-1][:top_k]

    # Find the documents corresponding to the top-k chunks
    retrieved_docs = [doc_chunk_mapping[idx] for idx in top_k_chunk_indices]

    # Count the number of times the relevant document appears in the top-k retrieved documents
    relevant_document_hits = sum(1 for doc_idx in retrieved_docs if doc_idx == relevant_idx)

    # Compute rank (based on first appearance of the relevant document)
    rank = None
    for idx, doc_idx in enumerate(retrieved_docs):
        if doc_idx == relevant_idx:
            rank = idx + 1
            break

    # Compute metrics
    is_hit = relevant_document_hits > 0  # Hit if any of the top-k chunks belong to the relevant document
    mrr = 0 if rank is None else 1 / rank

    return {
        "is_hit": is_hit,
        "mrr": mrr,
        "retrieved": retrieved_docs,
        "expected": relevant_idx
    }


def evaluate_model(df, model, query_column, query_emb_column, opinion_column, opinion_emb_column, top_k=5):
    # Step 1: Create embeddings for queries and opinions
    df = create_embeddings(df, model, query_column, query_emb_column)
    df = create_embeddings(df, model, opinion_column, opinion_emb_column)

    # Step 2: Evaluate retrieval for each query
    results_dict = {}
    for idx, query_row in df.iterrows():
        opinion_id = query_row["opinion_id"]
        query_emb = query_row[query_emb_column]
        doc_embs = df[opinion_emb_column]
        results_dict[opinion_id] = evaluate_retrieval(query_emb, doc_embs, idx, top_k=top_k)

    # Step 3: Convert results to DataFrame
    results_df = pd.DataFrame.from_dict(results_dict, orient='index').reset_index().rename(columns={'index': 'opinion_id'})

    # Step 4: Merge results with the original DataFrame
    final_df = df.merge(results_df, how="left", on="opinion_id")

    # Step 5: Calculate evaluation metrics (Hit Rate and MRR)
    model_name = model.name
    EVALS[model_name] = {}
    EVALS[model_name]["hit_rate"] = final_df["is_hit"].mean()
    EVALS[model_name]["mrr"] = final_df["mrr"].mean()

    return final_df

# Load the data

In [5]:
df = pd.read_csv("outputs/3.test.csv")
df = df[["opinion_id", "opinion_word_count", "opinion", "relevant_query_qstn"]]
df.head()

,opinion_id,opinion_word_count,opinion,relevant_query_qstn
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...


# Evaluate model: multi-qa-mpnet-base-dot-v1

In [6]:
%%time

model_name = "multi-qa-mpnet-base-dot-v1"
model = SentenceTransformer(f'sentence-transformers/{model_name}')
model.name = model_name
result1 = evaluate_model(df, model, "relevant_query_qstn", "query_embeddings", "opinion", "opinion_embeddings")

result1.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 1min 18s, sys: 2.68 s, total: 1min 20s
Wall time: 1min 29s


,opinion_id,opinion_word_count,opinion,relevant_query_qstn,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...,"[[-0.032940004, -0.29310903, -0.29131132, 0.07...","[[0.03615426, 0.23663479, -0.18826829, 0.06918...",True,1.0,"[0, 381, 125, 381, 381]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...,"[[-0.028817186, -0.054095395, -0.18098816, -0....","[[0.22053322, -0.062435195, 0.08577951, -0.176...",True,1.0,"[1, 1, 1, 170, 331]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...,"[[0.019929392, -0.15144193, -0.29583222, -0.15...","[[0.2780603, -0.36399046, -0.15213007, -0.1735...",False,0.0,"[165, 382, 425, 124, 50]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...,"[[-0.04887789, 0.10266887, -0.24811848, -0.137...","[[0.17971808, -0.11428497, -0.14335318, -0.207...",True,0.2,"[218, 439, 55, 44, 3]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...,"[[0.015778188, -0.17274271, -0.278399, -0.0880...","[[0.33654806, 0.28492162, -0.051344726, 0.3195...",True,1.0,"[4, 4, 4, 4, 4]",4


In [7]:
result1.to_csv("4a.eval/QA-512/3.result1.csv", index=False)
len(result1)

450

In [8]:
EVALS

{'multi-qa-mpnet-base-dot-v1': {'hit_rate': 0.8044444444444444,
  'mrr': 0.7302962962962963}}

# Evaluate model: all-mpnet-base-v2

In [9]:
%%time

model_name = "all-mpnet-base-v2"
model = SentenceTransformer(f'sentence-transformers/{model_name}')
model.name = model_name
result2 = evaluate_model(df, model, "relevant_query_qstn", "query_embeddings", "opinion", "opinion_embeddings")

result2.head()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 1min 1s, sys: 2.24 s, total: 1min 4s
Wall time: 1min 9s


,opinion_id,opinion_word_count,opinion,relevant_query_qstn,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...,"[[0.043355327, -0.02041288, 0.0140225915, -0.0...","[[-0.0080017205, 0.06487662, 0.02247683, 0.022...",True,1.0,"[0, 51, 381, 381, 51]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...,"[[0.012848754, 0.036895238, 0.03304124, -0.044...","[[0.024556143, 0.114506386, 0.013218337, -0.06...",True,1.0,"[1, 1, 349, 424, 1]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...,"[[0.013068957, -0.058598362, -0.009275695, -0....","[[0.045272153, -0.013880589, -0.0011564802, -0...",False,0.0,"[165, 382, 425, 23, 387]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...,"[[0.002592886, 0.0042303526, 0.0158242, -0.001...","[[0.023201818, 0.030269556, 0.0069102417, 0.00...",False,0.0,"[55, 144, 55, 4, 367]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...,"[[0.025586488, 0.0010452393, 0.012582706, -0.0...","[[0.035133556, 0.10087562, 0.023342265, 0.0482...",True,1.0,"[4, 4, 4, 4, 4]",4


In [10]:
result2.to_csv("4a.eval/QA-512/3.result2.csv", index=False)
len(result2)

450

In [11]:
EVALS

{'multi-qa-mpnet-base-dot-v1': {'hit_rate': 0.8044444444444444,
  'mrr': 0.7302962962962963},
 'all-mpnet-base-v2': {'hit_rate': 0.7688888888888888,
  'mrr': 0.6995555555555555}}

# Evaluate model: multi-qa-distilbert-cos-v1

In [12]:
%%time

model_name = "multi-qa-distilbert-cos-v1"
model = SentenceTransformer(f'sentence-transformers/{model_name}')
model.name = model_name
result3 = evaluate_model(df, model, "relevant_query_qstn", "query_embeddings", "opinion", "opinion_embeddings")

result3.head()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 36.6 s, sys: 1.95 s, total: 38.5 s
Wall time: 44.8 s


,opinion_id,opinion_word_count,opinion,relevant_query_qstn,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...,"[[-0.016788224, 0.008796586, -0.029689074, 0.0...","[[-0.034727223, -0.04516634, -0.011392552, 0.0...",True,1.0,"[0, 381, 381, 381, 125]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...,"[[-0.011966383, -0.014375327, -0.021743214, 0....","[[-0.020712376, -0.043643463, -0.0012547055, 0...",True,1.0,"[1, 1, 431, 328, 331]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...,"[[-0.044886865, 0.026831139, -0.013975966, 0.0...","[[-0.010372783, -0.050516944, -0.033840246, 0....",False,0.0,"[165, 425, 25, 25, 342]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...,"[[-0.034710567, -0.06026019, 0.0036644172, 0.0...","[[-0.068658866, -0.013247591, -0.016058907, 0....",False,0.0,"[428, 363, 428, 154, 137]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...,"[[-0.024807695, -0.004883751, -0.025458189, 0....","[[0.026483446, -0.028000247, -0.04994751, 0.00...",True,1.0,"[4, 4, 4, 4, 377]",4


In [13]:
result3.to_csv("4a.eval/QA-512/3.result3.csv", index=False)
len(result3)

450

In [14]:
EVALS

{'multi-qa-mpnet-base-dot-v1': {'hit_rate': 0.8044444444444444,
  'mrr': 0.7302962962962963},
 'all-mpnet-base-v2': {'hit_rate': 0.7688888888888888,
  'mrr': 0.6995555555555555},
 'multi-qa-distilbert-cos-v1': {'hit_rate': 0.7622222222222222,
  'mrr': 0.6759259259259258}}

# Evaluate model: all-distilroberta-v1

In [15]:
%%time

model_name = "all-distilroberta-v1"
model = SentenceTransformer(f'sentence-transformers/{model_name}')
model.name = model_name
result4 = evaluate_model(df, model, "relevant_query_qstn", "query_embeddings", "opinion", "opinion_embeddings")

result4.head()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 37.9 s, sys: 1.98 s, total: 39.9 s
Wall time: 46 s


,opinion_id,opinion_word_count,opinion,relevant_query_qstn,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",What are the contract terms determined by the ...,"[[-0.011537511, 0.00015323628, -0.017806258, -...","[[0.000111384004, -0.02229816, -0.023021718, -...",False,0.000000,"[381, 303, 51, 266, 381]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,What are the legal requirements for a claim of...,"[[-0.024155786, 0.0042835814, -0.009825836, 0....","[[0.014606625, -0.00021094125, 0.021554783, -0...",True,0.500000,"[331, 1, 1, 97, 395]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,What factors are considered in determining ine...,"[[0.0026988108, 0.045794923, 0.009991173, -0.0...","[[0.030920794, -0.03444464, -0.011262654, 0.00...",False,0.000000,"[165, 266, 425, 50, 387]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,What were the grounds for Roger Fincher's appe...,"[[0.015905123, -0.00059489877, 0.0024467243, 0...","[[0.03038813, 0.026680142, 0.016184725, 0.0171...",True,0.333333,"[144, 4, 3, 144, 233]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,What criteria did the Idaho Supreme Court use ...,"[[0.0052556726, 0.010727528, -0.018651739, 0.0...","[[-0.017508037, 0.01724882, 0.019661115, -0.02...",True,1.000000,"[4, 4, 4, 4, 4]",4


In [16]:
result4.to_csv("4a.eval/QA-512/3.result4.csv", index=False)
len(result4)

450

In [17]:
EVALS

{'multi-qa-mpnet-base-dot-v1': {'hit_rate': 0.8044444444444444,
  'mrr': 0.7302962962962963},
 'all-mpnet-base-v2': {'hit_rate': 0.7688888888888888,
  'mrr': 0.6995555555555555},
 'multi-qa-distilbert-cos-v1': {'hit_rate': 0.7622222222222222,
  'mrr': 0.6759259259259258},
 'all-distilroberta-v1': {'hit_rate': 0.7555555555555555,
  'mrr': 0.6718518518518519}}